<a href="https://colab.research.google.com/github/BaseKan/aiday_training_resources/blob/main/Cython/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%load_ext Cython
import numpy as np
from math import sin,tan,cos

# Fibonnaci

In [ ]:
%%cython

def c_fib_typed(int n):
  """Return the Fibonacci series up to n."""
  cdef int a, b, len_r
  cdef int r[1000]
  a = 0
  b = 1
  len_r = 0
  while b < n:
    r[len_r] = b
    len_r += 1
    a, b = b, a + b
        
  r_as_list  = [item for item in r[:len_r]]
  return r_as_list

In [ ]:
%%timeit
x = c_fib_typed(2000)

# Types en Arrays

In Cython is er een verschil tussen 32 en 64 bit ints en floats. Dit is in Python niet het geval. Je ints kunnen altijd zo groot zijn als je systeem aan kan.

In [ ]:
%%cython
cdef int i = 2147483647
print(i)
print(i+1)

In [ ]:
%%cython
cdef long l = 2147483647
print(l)
print(l+1)

Cython checkt niet of je over de grote van een array heen gaat als je hem op de C manier aanmaakt. Pas dus op hiermee!

In [ ]:
%%cython
cdef int a[3]
for i in range(3):
  a[i] = 1
print(a)
print(a[4])
print(a[5])
print(a[200])

# Cython and Numpy Arrays

Memory Views geven je efficient toegang tot het onderliggende geheugen van bijvoorbeeld Numpy arrays.

In [ ]:
%%cython
import numpy as np

arr = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(arr)

cdef int [:, :] arr_view = arr
print(np.asarray(arr_view))

In [ ]:
%%cython 
import numpy as np

cdef int [:, :] arr_view = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(np.asarray(arr_view))
print(np.asarray(arr_view[1,1]))
arr_view[1,1] = 10
print(np.asarray(arr_view[1,1]))
arr_view[:,:] = 5
print(np.asarray(arr_view))


Let wel op! De numpy array moet expliciet hetzelfde type hebben als de Memory View om ze samen te gebruiken. Vergeet dus niet de types te definiëren als je je numpy array aanmaakt.

## Arrays en for loops

In [ ]:
x = np.arange(100, dtype=np.dtype("i")).reshape((10, 10))
y = np.full((10,10), 2, dtype=np.dtype("i"))

In [ ]:
def array_op(x,y):
  result = np.zeros(x.shape)
  for i in range(x.shape[0]):
      for j in range(x.shape[1]):
          result[i,j] = x[i,j] * y[i,j] + i
  return result
    

In [ ]:
%timeit result = array_op(x,y)

In [ ]:
%%cython 
import numpy as np

def c_array_op(x, y):
    result = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [ ]:
%timeit c_array_op(x,y)

In [ ]:
%%cython 
import numpy as np

def typed_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [ ]:
%timeit typed_array_op(x,y)

In [ ]:
%%cython 
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def typed_unsafe_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result


In [ ]:
%timeit typed_unsafe_array_op(x,y)

# Opdracht: np.vectorize naar Cython

De onderstaande code is de oplossing van één van de opdrachten van de Optimalisatie kennissessie. Met np.vectorize is die al een stuk sneller gemaakt. 

In [ ]:
def complicated_calculation(x,y):
  if x > 0.5*y and y < 0.3:
      res = sin(x-y)
  elif x < 0.5*y:
      res = tan(x+y)
  elif x > 0.2*y:
      res = sin(x)*np.sin(y)
  else:
      res = cos(x/(0.1+abs(y)))
  return res

In [ ]:
def get_results_fast(x,y):
  return np.vectorize(complicated_calculation)(x,y)

In [ ]:
x = np.random.randn(int(1e6))
y = np.random.randn(int(1e6))

In [ ]:
%timeit res_fast = get_results_fast(x, y)

Het kan echter nog een stuk sneller in Cython. Vul de onderstaande code aan. In plaats van np.vectorize kun je een for loop gebruiken in Cython. Ook kan het een stuk beter door types toe te voegen. 

Een eerste stap is al gemaakt door de sin, cos, tan en abs operaties te vervangen door equivalente operaties in C.

In [ ]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np
from libc.math cimport sin, cos, tan, fabs

def complicated_calculation(x,y):
  if x > 0.5*y and y < 0.3:
      res = sin(x-y)
  elif x < 0.5*y:
      res = tan(x+y)
  elif x > 0.2*y:
      res = sin(x)*sin(y)
  else:
      res = cos(x/(0.1+fabs(y)))
  return res

def c_get_results_fast(x, y):
  # TODO: For loop toevoegen
  return res

In [ ]:
%timeit res_fast = c_get_results_fast(x, y)

# Cython and Pandas

We downloaden eerst wat data...

In [ ]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'weatherAUS.zip' 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd&confirm='$(<confirm.txt)
!unzip weatherAUS.zip
!rm -f confirm.txt cookies.txt weatherAUS.zip

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('weatherAUS.csv')

In [ ]:
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype == 'float64' else x,
              axis=0)
df.Date = pd.to_datetime(df.Date)

df = df.sort_values('Date').reset_index(drop=True)
df.head()

In [ ]:
def average_value(values):
  return np.mean(values)


In [ ]:
%timeit df.head(100).apply(lambda x: average_value(x[11:13]), axis=1)


In [ ]:
%%cython
import numpy as np

def c_average_value(values):
  return np.mean(values)


In [ ]:
%timeit df.head(100).apply(lambda x: c_average_value(x[11:13]), axis=1)

We gaan weer werken met Memory Views. Let wel op: Memory Views zijn vrij specifiek voor numpy arrays geïmplementeerd. Een deel van een rij ophalen uit een pandas dataframe geeft een `Series` object terug. Deze moet je dus eerst omzetten naar numpy.

In [ ]:
%%cython
import numpy as np
cimport numpy as np

def c_average_value_typed(double[:] values):
  return np.mean(values)

In [ ]:
%timeit df.head(100).apply(lambda x: c_average_value_typed(x[11:13].to_numpy(dtype=np.dtype('d'))), axis=1)

`apply` is nu de bottleneck, dus die gaan we wegwerken.


In [ ]:
%%cython --annotate
cimport cython
cimport numpy as np
import numpy as np

cdef double c_average_value_typed(double[:] values):
  return np.mean(values)

def c_apply_average_value(double[:,:] df_cols):
  n = df_cols.shape[0]
  cdef double[:] res = np.empty(n, dtype=np.dtype('d'))
  for i in range(n):
    res[i] = c_average_value_typed(df_cols[i])

  return res

In [ ]:
%timeit c_apply_average_value(df.head(100)[df.columns[11:13]].to_numpy(dtype=np.dtype('d'))) 

# Opdracht: 

Zet de snelle code van Lisa in Cython.

In [ ]:
df['AvgTemp'] = df[['MinTemp', 'MaxTemp']].mean(axis=1)

In [ ]:
def classify_temperature(df):
  bins = [df.AvgTemp.describe()['min'], df.AvgTemp.describe()['25%'], df.AvgTemp.describe()['75%'], df.AvgTemp.describe()['max']]
  labels = ['cold','average','hot']
  df['TempType'] = pd.cut(df['AvgTemp'], bins=bins, labels=labels)
  return df

In [ ]:
%%timeit
classify_temperature(df)

Een deel is al gedaan voor je.

Hint: Het type van een python string is `object`.

In [ ]:
%%cython
def c_classify_temperature(double[:] avg_temp):
  cold = np.quantile(avg_temp, 0.25)
  hot = np.quantile(avg_temp, 0.75)
  # YOUR CODE HERE
  

# Extra materiaal

Cython documentatie: https://cython.readthedocs.io/en/latest/

Pandas optimalisatie: https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html

Just in time compiler: https://numba.pydata.org/

Julia: https://julialang.org/

